In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm

# from jax import random as jrandom
# set the PYTHONPATH correctly for imports
path = os.path.abspath('..')
sys.path.insert(0, path)

# from data_pytorch import get_dataloaders
# from searchless_chess.src import config as config_lib
# from searchless_chess.src.data_loader import build_data_loader

# from searchless_chess.src import transformer
# from searchless_chess.src import tokenizer
# from searchless_chess.src import utils
# from searchless_chess.src import training_utils

In [2]:
import chessformers

In [3]:
# pip install torch transformers huggingface_hub chess
import torch

from chess_former.model 

model = Transformer.from_pretrained("kaupane/ChessFormer-SL")
model.eval()

fens = ["r1bqkbnr/pppp1ppp/2n5/4p3/1P2P3/P1N5/2PP1PPP/R1BQKBNR b KQkq b3 0 3"]
repetitions = torch.tensor([1])  # repetition count token if needed
with torch.no_grad():
    move_logits, position_value = model(fens, repetitions)
print(float(position_value[0]))   # e.g., value ~[-1, 1]


SyntaxError: invalid syntax (3845541900.py, line 4)

# Load chessformer model

In [4]:
!ls ..

 chessformer_epoch_13.pth		 searchless_chess
 chessformers				 src
 data					 train_student_model.py
 download_chessformer_model.sh		'view?usp=drive_link'
 plan_searchless_chess_distillation.md


In [3]:
import torch
import chess
from chessformers.chessformers.model import Transformer # Import the model class

# --- 1. Define Model Parameters ---
# These parameters MUST match the ones used to train the saved model.
# (These are the defaults from the chessformers repository)
d_model = 512
n_head = 8
n_layers = 8
d_k = 64
d_v = 64
d_ff = 2048
dropout = 0.1
max_len = 128  # Max sequence length

# --- 2. Instantiate the Model ---
# Create an instance of the model architecture
print("Initializing model architecture...")
model = Transformer(
    n_layers=n_layers,
    n_head=n_head,
    d_model=d_model,
    d_k=d_k,
    d_v=d_v,
    d_ff=d_ff,
    dropout=dropout,
    max_len=max_len
)

# --- 3. Load the Pretrained Weights ---
# Define the path to your downloaded .pth file
weights_path = './model/chessformer_epoch_13.pth'

try:
    print(f"Loading pretrained weights from {weights_path}...")
    # Load the state dictionary (the weights)
    # Use map_location to load onto CPU if you don't have a GPU
    state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
    
    # Load the weights into the model instance
    model.load_state_dict(state_dict)
    
    # --- 4. Set Model to Evaluation Mode ---
    # This is crucial! It disables dropout and batch norm layers.
    model.eval()
    
    print("\nSuccessfully loaded pretrained model!")
    print(f"Model has {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

    # Now the 'model' object is ready to be used for inference
    # For example, you would typically:
    # 1. Get the current board state (from python-chess)
    # 2. Tokenize/encode the board state into tensors
    # 3. Feed the tensors to model(input_tensors)
    # 4. Decode the model's output to get a move

except FileNotFoundError:
    print(f"Error: Weights file not found at {weights_path}")
    print("Please download 'chessformer_epoch_13.pth' and place it in the './model/' directory.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

ModuleNotFoundError: No module named 'chessformers.model'

# Dataloaders

In [3]:
from searchless_chess.src import config as config_lib
from searchless_chess.src.data_loader import build_data_loader
import jax.numpy as jnp  # or numpy as np

# Build config
cfg = config_lib.DataConfig(
    policy='state_value',         # <- important
    split='train',                # 'train' or 'test'
    batch_size=10,
    num_return_buckets=128,       # label bins
    shuffle=True,
    seed=0,
    worker_count=2,
    num_records=None,             # or an int to cap
)

loader = build_data_loader(cfg)

for sequences, loss_mask in loader:
    # sequences shape: (B, tokenizer.SEQUENCE_LENGTH + 1)
    # last token is the return-bucket label
    X_tokens = sequences[:, :-1]      # (B, L)  board tokens
    y_bins   = sequences[:, -1]       # (B,)    int labels in [0..127]
    # loss_mask masks everything except the last token (loss on y only)
    # ... your code (train / save / inspect) ...
    break  # demo: take one batch


In [6]:
X_tokens.shape

(10, 77)

In [9]:
config = config_lib.DataConfig(
    policy='state_value',
    split='train',
    batch_size=4096,  # adjust as needed
    shuffle=False,  # set to False to maintain order
    worker_count=0,
    seed=42,
    num_return_buckets=128  # adjust based on your config
)
data_loader = build_data_loader(config)

In [10]:
batch = next(iter(data_loader))

In [16]:
batch[0]

array([[ 11,  30,  30, ...,   9,  30,  18],
       [ 11,  30,  30, ...,   1,  30, 119],
       [ 11,  30,  30, ...,   5,  30,  11],
       ...,
       [ 29,  30,  30, ...,   4,  30, 122],
       [ 29,  30,  30, ...,   7,  30, 122],
       [ 11,  30,  30, ...,   8,  30,  57]])

# Data size

In [4]:
train_data_path = os.path.join(os.getcwd(), 'data', 'train')
test_data_path = os.path.join(os.getcwd(), 'data', 'test')

batch_size = 1000

In [5]:
train_loader, val_loader = get_dataloaders(batch_size=batch_size)

In [6]:
batch = next(iter(train_loader))
print(batch)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_5404/1444793255.py", line 1, in <module>
    batch = next(iter(train_loader))
  File "/teamspace/studios/this_studio/distill-chess/src/data_pytorch.py", line 32, in __iter__
    features_tensor = torch.from_numpy(batch_np).float().pin_memory()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/pyt

In [5]:
x = batch[0]

In [6]:
x[0]

tensor([29., 20., 30., 11., 22., 30., 20., 21., 30., 18., 30., 30., 30., 11.,
        18., 18., 18., 19., 30., 18., 18., 30., 19., 30., 30., 30., 18., 30.,
        30., 18., 30., 30., 30., 30., 30., 30., 30., 23., 30., 30., 30., 30.,
        30., 25., 23., 30., 25., 23., 30., 23., 23., 23., 24., 24., 23., 30.,
        23., 26., 30., 30., 27., 30., 28., 30., 26., 30., 30., 30., 30., 30.,
        30.,  0., 30., 30.,  9., 30., 30.])

In [9]:
batch[1]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

# Numpy dataset

In [9]:
NP_DATASETS_PATH = '../data/numpy_arrays'

In [10]:
!ls {NP_DATASETS_PATH}

state_value_test.npy_masks.npy	    state_value_train.npy_sequences.npy
state_value_test.npy_sequences.npy  state_value_train_masks.npy
state_value_train.npy_masks.npy     state_value_train_sequences.npy


In [11]:
train_state_value = np.load(os.path.join(NP_DATASETS_PATH, 'state_value_train.npy_sequences.npy'), mmap_mode='r')

In [12]:
train_state_value[0]

memmap([11, 30, 30, 30, 30, 30, 20, 21, 30, 30, 22, 30, 30, 30, 18, 18,
        30, 30, 30, 30, 30, 11, 30, 30, 18, 30, 30, 30, 30, 27, 30, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 23, 30, 30, 30, 30, 30,
        30, 30, 30, 23, 30, 30, 23, 23, 23, 26, 30, 30, 30, 26, 30, 28,
        30, 30, 30, 30, 30, 30, 30,  0, 30, 30,  2,  9, 30, 18],
       dtype=uint8)

In [13]:
train_state_value.shape

(4096000, 78)

# Load searchless_chess model 

In [1]:
import os
import sys

path = os.path.abspath('..')
sys.path.insert(0, path)

In [2]:
import numpy as np
import jax.random as jrandom
import orbax.checkpoint as ocp
from searchless_chess.src import transformer, tokenizer

def load_state_value(dir_):
    cfg = transformer.TransformerConfig(
        vocab_size=len(tokenizer._CHARACTERS),
        output_size=128,
        pos_encodings=transformer.PositionalEncodings.LEARNED,
        max_sequence_length=tokenizer.SEQUENCE_LENGTH + 1,
        num_heads=8, num_layers=8, embedding_dim=256,
        apply_post_ln=True, apply_qk_layernorm=False, use_causal_mask=False,
    )
    predictor = transformer.build_transformer_predictor(config=cfg)

    dummy = predictor.initial_params(
        rng=jrandom.PRNGKey(0),
        targets=np.zeros((1, 1), dtype=np.uint32),
    )

    # Wrap for Orbax (Orbax expects a top-level key; ckpt itself is the *inner* tree)
    dummy_wrapped = {"params": dummy}

    ckpt = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
    restore_args = ocp.checkpoint_utils.construct_restore_args(dummy_wrapped)
    restored_wrapped = ckpt.restore(dir_, restore_args=restore_args)

    params = restored_wrapped["params"]  # unwrap back to Haiku tree
    return predictor, params


AttributeError: 'Config' object has no attribute 'define_bool_state'

In [3]:
predictor, params = load_state_value('../searchless_chess/checkpoints/9M_state_value/')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_89455/2879037319.py", line 1, in <module>
    predictor, params = load_state_value('../searchless_chess/checkpoints/9M_state_value/')
  File "/tmp/ipykernel_89455/1907418656.py", line 28, in load_state_value
    restored = ckpt.restore(dir_, restore_args=restore_args)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/orbax/checkpoint/checkpointer.py", line 165, in restore
    restored = self._restore_with_args(directory, *args, **kwargs)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/orbax/checkpoint/checkpointer.py", line 103, in _restore_with_args
    restored = self._handler.restore(directory, args=ckpt_args)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/orbax/c

In [28]:
print(list(dummy_params.keys()))

['embed', 'embed_1', 'layer_norm', 'multi_head_dot_product_attention/linear', 'multi_head_dot_product_attention/linear_1', 'multi_head_dot_product_attention/linear_2', 'multi_head_dot_product_attention/linear_3', 'layer_norm_1', 'linear', 'linear_1', 'linear_2', 'layer_norm_2', 'multi_head_dot_product_attention_1/linear', 'multi_head_dot_product_attention_1/linear_1', 'multi_head_dot_product_attention_1/linear_2', 'multi_head_dot_product_attention_1/linear_3', 'layer_norm_3', 'linear_3', 'linear_4', 'linear_5', 'layer_norm_4', 'multi_head_dot_product_attention_2/linear', 'multi_head_dot_product_attention_2/linear_1', 'multi_head_dot_product_attention_2/linear_2', 'multi_head_dot_product_attention_2/linear_3', 'layer_norm_5', 'linear_6', 'linear_7', 'linear_8', 'layer_norm_6', 'multi_head_dot_product_attention_3/linear', 'multi_head_dot_product_attention_3/linear_1', 'multi_head_dot_product_attention_3/linear_2', 'multi_head_dot_product_attention_3/linear_3', 'layer_norm_7', 'linear_9',

In [21]:
inner_restore_args

{'embed': {'embeddings': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=SingleDeviceSharding(device=CpuDevice(id=0), memory_kind=unpinned_host), global_shape=(31, 256), shape=(31, 256), strict=True)},
 'embed_1': {'embeddings': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=SingleDeviceSharding(device=CpuDevice(id=0), memory_kind=unpinned_host), global_shape=(78, 256), shape=(78, 256), strict=True)},
 'layer_norm': {'offset': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=SingleDeviceSharding(device=CpuDevice(id=0), memory_kind=unpinned_host), global_shape=(256,), shape=(256,), strict=True),
  'scale': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=SingleDeviceSharding(device=CpuDevice(id=0), memory_kind=unpinned_host), global_shape=(

In [16]:
dummy_params

{'embed': {'embeddings': Array([[-0.03770083, -0.03440559,  0.00392136, ..., -0.00755102,
           0.00618673,  0.01569704],
         [ 0.00212923,  0.01515061,  0.01002361, ...,  0.02043889,
          -0.00314988, -0.00278346],
         [-0.00101512,  0.03413207,  0.01374406, ...,  0.00516759,
          -0.02214212, -0.02639753],
         ...,
         [ 0.01131086,  0.00149996,  0.01039171, ..., -0.00521467,
           0.03148041, -0.0056405 ],
         [ 0.01620454,  0.02170458,  0.03577229, ..., -0.0068321 ,
          -0.03540927,  0.0082824 ],
         [-0.00512792,  0.03593934,  0.00528113, ..., -0.0077195 ,
          -0.00706294,  0.00413409]], dtype=float32)},
 'embed_1': {'embeddings': Array([[-1.1635892 , -0.38238966, -1.0456824 , ...,  1.694646  ,
          -0.38262197, -0.28035673],
         [-0.38238287,  0.841924  ,  0.34093148, ..., -0.0853501 ,
           0.6698055 , -0.95979536],
         [ 0.0313274 , -0.85554594,  0.16286285, ..., -0.73611283,
           0.01371196

In [11]:
policy = 'action_value'
num_return_buckets = 128

match policy:
  case 'action_value':
    output_size = num_return_buckets
  case 'behavioral_cloning':
    output_size = utils.NUM_ACTIONS
  case 'state_value':
    output_size = num_return_buckets
  case _:
    raise ValueError(f'Unknown policy: {policy}')

predictor_config = transformer.TransformerConfig(
    vocab_size=utils.NUM_ACTIONS,
    output_size=output_size,
    pos_encodings=transformer.PositionalEncodings.LEARNED,
    max_sequence_length=tokenizer.SEQUENCE_LENGTH + 2,
    num_heads=4,
    num_layers=4,
    embedding_dim=64,
    apply_post_ln=True,
    apply_qk_layernorm=False,
    use_causal_mask=False,
)

predictor = transformer.build_transformer_predictor(config=predictor_config)

In [7]:
!ls ../searchless_chess/checkpoints/

136M  270M  9M	9M_behavioral_cloning  9M_state_value  download.sh


In [12]:

# @title Load the predictor parameters

checkpoint_dir = f'../searchless_chess/checkpoints/9M/'
dummy_params = predictor.initial_params(
    rng=jrandom.PRNGKey(0),
    targets=np.zeros((1, 1), dtype=np.uint32),
)
params = training_utils.load_parameters(
    checkpoint_dir=checkpoint_dir,
    params=dummy_params,
    use_ema_params=True,
    step=-1,
)

1
[6400000]


ValueError: Dict key mismatch; expected keys: ['embed', 'embed_1', 'layer_norm', 'layer_norm_1', 'layer_norm_10', 'layer_norm_11', 'layer_norm_12', 'layer_norm_13', 'layer_norm_14', 'layer_norm_15', 'layer_norm_16', 'layer_norm_2', 'layer_norm_3', 'layer_norm_4', 'layer_norm_5', 'layer_norm_6', 'layer_norm_7', 'layer_norm_8', 'layer_norm_9', 'linear', 'linear_1', 'linear_10', 'linear_11', 'linear_12', 'linear_13', 'linear_14', 'linear_15', 'linear_16', 'linear_17', 'linear_18', 'linear_19', 'linear_2', 'linear_20', 'linear_21', 'linear_22', 'linear_23', 'linear_24', 'linear_3', 'linear_4', 'linear_5', 'linear_6', 'linear_7', 'linear_8', 'linear_9', 'multi_head_dot_product_attention/linear', 'multi_head_dot_product_attention/linear_1', 'multi_head_dot_product_attention/linear_2', 'multi_head_dot_product_attention/linear_3', 'multi_head_dot_product_attention_1/linear', 'multi_head_dot_product_attention_1/linear_1', 'multi_head_dot_product_attention_1/linear_2', 'multi_head_dot_product_at

In [2]:
output_size = 128  # number of return buckets
model_size = "9M"
policy = "state_value"
ckpt_dir = '../searchless_chess/checkpoints/9M_state_value' #f"./checkpoints/{model_size}/state_value" 

In [3]:
cfg = transformer.TransformerConfig(
      vocab_size=utils.NUM_ACTIONS,
      output_size=output_size,
      pos_encodings=transformer.PositionalEncodings.LEARNED,
      max_sequence_length=tokenizer.SEQUENCE_LENGTH + 2,
      num_heads=4,
      num_layers=4,
      embedding_dim=64,
      apply_post_ln=True,
      apply_qk_layernorm=False,
      use_causal_mask=False,
  )

In [4]:
predictor = transformer.build_transformer_predictor(cfg)

In [18]:
!ls ../searchless_chess/checkpoints/9M_state_value/	 #/#9M/state_value

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


_METADATA  checkpoint  manifest.0000000000000001  manifest.ocdbt
_sharding  d	       manifest.0000000000000002  ocdbt.process_0


In [20]:
import jax
import jax.numpy as jnp
import orbax.checkpoint as ocp
from flax.training import orbax_utils

# 1) Build your model and get a params structure (shapes/dtypes only)
#    Use a real sample_input shaped like your training data.
variables = dummy_params #model.init(jax.random.PRNGKey(0), sample_input)
params_like = variables["params"]  # flax variables dict

# 2) Prepare restore args for Orbax
restore_args = orbax_utils.restore_args_from_target({"params_ema": params_like})

# 3) Restore from the OCDBT checkpoint directory (the one that contains manifest.*)
ckpt_dir = "../searchless_chess/checkpoints/9M_state_value/0"

checkpointer = ocp.Checkpointer(
    ocp.CompositeCheckpointHandler({
        "params_ema": ocp.PyTreeCheckpointHandler()
    })
)

restored = checkpointer.restore(
    ckpt_dir,
    item={"params_ema": params_like},
    restore_kwargs={"restore_args": restore_args},
)

ema_params = restored["params_ema"]  # ← use these in your model.apply / distillation


KeyError: 'params'

In [15]:
!ls ../searchless_chess/checkpoints/9M_state_value	

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


_METADATA  checkpoint  manifest.0000000000000001  manifest.ocdbt
_sharding  d	       manifest.0000000000000002  ocdbt.process_0


In [16]:
# @title Load the predictor parameters

checkpoint_dir = '../searchless_chess/checkpoints/9M_state_value'
dummy_params = predictor.initial_params(
    rng=jrandom.PRNGKey(0),
    targets=np.zeros((1, 1), dtype=np.uint32),
)
params = training_utils.load_parameters(
    checkpoint_dir=checkpoint_dir,
    params=dummy_params,
    use_ema_params=True,
    step=-1,
)

1
[0]


FileNotFoundError: Checkpoint at ../searchless_chess/checkpoints/9M_state_value/0/params_ema not found.

In [19]:
predictor.predict#((train_state_value[0][:-1]),rng=jnp.random.PRNGKey(0))

<function haiku._src.transform.without_state.<locals>.apply_fn(params: Optional[collections.abc.Mapping[str, collections.abc.Mapping[str, jax.Array]]], rng: Union[jax.Array, int, NoneType], targets: jax.Array, *, config: searchless_chess.src.transformer.TransformerConfig = TransformerConfig(seed=1, vocab_size=1968, output_size=128, embedding_dim=64, num_layers=4, num_heads=4, use_causal_mask=False, emb_init_scale=0.02, pos_encodings=<PositionalEncodings.LEARNED: 2>, max_sequence_length=79, widening_factor=4, apply_qk_layernorm=False, apply_post_ln=True)) -> jax.Array>

In [ ]:
# @title Create the engine

predict_fn = neural_engines.wrap_predict_fn(predictor, params, batch_size=1)
_, return_buckets_values = utils.get_uniform_buckets_edges_values(
    num_return_buckets
)

neural_engine = neural_engines.ENGINE_FROM_POLICY[policy](
    return_buckets_values=return_buckets_values,
    predict_fn=predict_fn,
    temperature=0.005,
)